#https://www.youtube.com/watch?v=V1VOdoEFaDw
#https://colab.research.google.com/drive/1JwZ_nWhBUFbrzJnHKmyd0qKJ3gVt5lCe?usp=sharing#scrollTo=Aw5t

# Vision-RAG - ColPali

In [ ]:
""" !pip install byaldi """

In [ ]:
""" !sudo apt-get install poppler-utils """

In [1]:
import base64
import os
# os.environ["HF_TOKEN"] = "" # to download the ColPali model from Hugging Face
from byaldi import RAGMultiModalModel

ModuleNotFoundError: No module named 'byaldi'

In [25]:
from transformers import AutoModel
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

RAG = RAGMultiModalModel.from_pretrained(
    "vidore/colpali-v1.3",
    #torch_dtype="bfloat16",
    device=device,   # automatically maps across AMD GPUs
    verbose=1
)

In [12]:
print(os.listdir("image/ColPali"))

In [26]:
RAG.index(
    input_path="image/ColPali",
    index_name="attention",
    store_collection_with_index=True,
    overwrite=True
)

In [27]:
from IPython.display import display
from PIL import Image
import io

In [28]:
question = "What is the net profit for Nike?"

In [29]:
results = RAG.search(question, k=1)

In [30]:
results

In [17]:
image_bytes = base64.b64decode(results[0].base64)

In [31]:
def show_results(question):
  results = RAG.search(question, k=1)
  image_bytes = base64.b64decode(results[0].base64)
  # Convert bytes to image using Pillow
  image = Image.open(io.BytesIO(image_bytes))

  # Display the image
  display(image)
  return image

In [32]:
# os.environ["OPENAI_API_KEY"] = 

In [33]:
import openai
import io

In [34]:
def answer_gpt4o(question, img):
    import io
    import base64
    import openai

    # Convert PIL image to bytes
    img_byte_arr = io.BytesIO()
    img.save(img_byte_arr, format='PNG')
    img_bytes = img_byte_arr.getvalue()

    # Encode image as base64 string
    img_b64 = base64.b64encode(img_bytes).decode("utf-8")

    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": f"Answer the question based on the following image. Don't use markdown. Please provide enough context for your answer.\n\nQuestion: {question}"
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{img_b64}"
                        }
                    }
                ]
            }
        ],
        max_tokens=1024
    )
    print("LLM Answer:", response.choices[0].message.content)


In [35]:
img = show_results(question)
answer_gpt4o(question, img)

In [ ]:
import ollama

In [41]:
def answer_ollama(question, img):
	res = ollama.chat(
			model="gemma3:12b",
			messages=[
				{
					'role': 'user',
					'content': question,
					'images': img
				}
			]
		) 
	print("LLM Answer:", res['message']['content'])
	


In [42]:
img = show_results(question)
answer_gpt4o(question, img)

In [ ]:
# Define the query
question = "What are the 3 largest acquisitions from Google?"
show_results(question)

In [ ]:
# Define the query
question = "What would be the net profit of Tesla without interest?"
show_results(question)

In [ ]:
question = "Is GenAI a good business for consulting companies?"
show_results(question)